Thymio mini project

In [1]:
!pip install --upgrade tdmclient

In [2]:
# import libraries
import time
from tdmclient import ClientAsync
import numpy as np
import math

# import project files
import constants as cst
import extended_Kalman_filter as eKf
import vision
import geometry as geo


-1.5707963267948966


In [3]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node 101c2fd7-1c04-4e4c-911d-5b3f3b0c1cc6

In [4]:
# Set the motor speeds
async def stop_robot():
    """
    Stop the robot. Set the motor speed of each wheel to zero.
    """
    await node.set_variables({
            "motor.left.target": [0],
            "motor.right.target": [0],
        })

async def move_robot(left_speed, right_speed):
    """
    Turn the robot wheels to the given speeds.
    Parameter: - left_speed    : left wheel speed in [thymio speed]
               - right_speed   : right wheel speed in [thymio speed]
    """
    await node.set_variables({
            "motor.left.target": [left_speed],
            "motor.right.target": [right_speed],
        })

In [5]:
# Get the proximity sensor values
async def get_prox_sensors():
    """
    Get the sensor measurements of a the proximity sensors.
    Returns: - prox_sensors    : numpy array of fron proximity sensors measurements
             - detected        : boolean, true if any sensors measured a higher value than threshold
    """
    
    await node.wait_for_variables({str('prox.horizontal')})
    prox_sensors = np.asarray(list(node['prox.horizontal']))
    prox_sensors = prox_sensors[0:5]

    prox_sensors_bool = (prox_sensors >= np.ones(5) * 50) #cst.THRESHOLD_PROX_SENSOR
    detected = False
    if(any(prox_sensors_bool)):
        detected = True

    return prox_sensors, detected

# Get the motor speed measurement values
async def get_motor_speed_meas():
    """
    Get the motor speed measurements
    Returns: - speed_left_meas    : left wheel speed in [thymio speed]
             - speed_right_meas   : right wheel speed in [thymio speed]
    """
    
    await node.wait_for_variables({str('motor.left.speed')})
    speed_left_meas = node['motor.left.speed']
    await node.wait_for_variables({str('motor.right.speed')})
    speed_right_meas = node['motor.right.speed']

    return speed_left_meas, speed_right_meas

In [6]:
# test thymio
# move the robot forward and get prox values.
await move_robot(50, 50)
dt = 0
old_time = time.time()
while(dt < 2):
    y,d = await get_prox_sensors()
    print(y)
    print(d)
    await client.sleep(0.1)
    dt = time.time() - old_time
await stop_robot()

[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False


In [9]:

# state variables
global_state = 'rotation'

### initialisation
# vision (détection robot, obstacles, goal)
#obstacles, goal_centroid, robot_centroid, robot_direction = vision.vision_obstacles_positions()
# global nav

robot, robot_direction = vision.vision_robot()
robot_orientation = np.arctan2(robot_direction[0], robot_direction[1])
robot = robot * 0.308

intermediate_goals = [(0, 10), (10, 10), (10, 0), (0,0)]
#robot = [0.0,0.0]
#robot_orientation = 0
robot_speed = [0.0,0.0]


Kfilter = eKf.Kalman(robot[0], robot[1], robot_orientation)


old_time = 0.0

iter = 0
while(True):
    print(iter)
    if intermediate_goals == []:
        await stop_robot()
        print("stop robot")
        break

    # vision et gobale toutes les n iterations (i modulo n == 0)
    
    # get robot sensor values
    #prox_sensors, detected = await get_prox_sensors()
    # test if local (with proximity)
    #if(detected):
    #    global_state = 'local_avoidance'
    
    if iter == 0:                   # first time
        old_time = time.time()
    dt = time.time() - old_time     # get time difference for Kalman
    # print("dt: ", dt)
    # Kalman (with wheel speed and camera position)
    old_time = time.time()

    #robot[0] = robot[0] + dt * ((robot_speed[0] + robot_speed[1])/(2.0*18.1818)) * math.cos(robot_orientation)
    #robot[1] = robot[1] + dt * ((robot_speed[0] + robot_speed[1])/(2.0*18.1818)) * math.sin(robot_orientation)
    #robot_orientation = robot_orientation + dt * (robot_speed[0] - robot_speed[1])/(4.0*cst.WHEELS_DIST)
    #print(robot[0]," ", robot[1]," ", robot_orientation)
    
    Kfilter.dt_update(dt)
    speed_left_meas, speed_right_meas = await get_motor_speed_meas()
    Kfilter.measurement_wheels(speed_left_meas / 18.1818, speed_right_meas / 18.1818)
    if(iter % 10 == 0):
        robot_centroid, robot_direction = vision.vision_robot()
        robot_centroid = robot_centroid * 0.308
        robot_angle = np.arctan2(robot_direction[0], robot_direction[1])
        Kfilter.measurement_position(robot_centroid[0], robot_centroid[1], robot_angle)
    Kfilter.Kalman_filter()
    print("Kalman: ", Kfilter.Mu)

    if global_state == 'rotation':
        angle = Kfilter.Mu[2] - np.arctan2(intermediate_goals[0][1] - Kfilter.Mu[1], intermediate_goals[0][0] - Kfilter.Mu[0])
        #angle = robot_orientation - np.arctan2(intermediate_goals[0][1] - robot[1], intermediate_goals[0][0] - robot[0])
        print("rotation")
        #print("rotation :", robot_orientation, np.arctan2(intermediate_goals[0][1] - robot[1], intermediate_goals[0][0] - robot[0]))
        
        if abs(angle) < 0.1:
            global_state = 'translation'
            continue

        # call rotation fct
        if(angle > 0):
            robot_speed = [-30, 30]
        else:
            robot_speed = [30, -30]
        await move_robot(robot_speed[0], robot_speed[1])
        

    elif global_state == 'translation':
        print("translation ", np.sqrt((intermediate_goals[0][0] - Kfilter.Mu[0])**2 + (intermediate_goals[0][1] - Kfilter.Mu[1])**2))
        #print("translation: ", robot[0], " ", robot[1], "  ", math.dist(intermediate_goals[0], robot))
        # if goal reached
            # exit
        #if abs(intermediate_goals[0][0] - Kfilter.Mu[0]) < 1 and abs(intermediate_goals[0][1] - Kfilter.Mu[1]) < 1:
        if np.sqrt((intermediate_goals[0][0] - Kfilter.Mu[0])**2 + (intermediate_goals[0][1] - Kfilter.Mu[1])**2) < 5:
        #if abs((intermediate_goals[0][0] - robot[0])**2 - (intermediate_goals[0][1] - robot[1])**2) < 2:
            intermediate_goals.pop(0)
            global_state = 'rotation'
            continue
        
        # call translation fct
        robot_speed = [70, 70]
        await move_robot(robot_speed[0], robot_speed[1])

    elif global_state == 'local_avoidance':
        # call local_avoidance fct (= add obstacle on global map and recompute global nav)
        
        global_state = 'rotation'
    time.sleep(0.05)

    iter = iter + 1

0
Vecteur direction du robot : -2147483648
Centroid du robot : [-2147483648 -2147483648]
Kalman:  [[-1.43167963e+09]
 [-1.43167963e+09]
 [-1.57082251e+00]
 [ 8.52501877e-01]
 [-7.97501756e-01]]
rotation
1
Kalman:  [[-1.43167963e+09]
 [-1.43167963e+09]
 [-2.25268294e-01]
 [ 1.19900189e+00]
 [-1.14400181e+00]]
rotation
2
Kalman:  [[-1.43167963e+09]
 [-1.43167963e+09]
 [-1.71798211e-01]
 [ 1.34115555e+00]
 [-1.28615549e+00]]
rotation
3
Kalman:  [[-1.43167963e+09]
 [-1.43167963e+09]
 [-1.28610619e-01]
 [ 1.46397227e+00]
 [-1.47691350e+00]]
rotation
4
Kalman:  [[-1.43167963e+09]
 [-1.43167963e+09]
 [-9.28502110e-02]
 [ 1.51095668e+00]
 [-1.48191170e+00]]
rotation
5
Kalman:  [[-1.43167963e+09]
 [-1.43167963e+09]
 [-5.19292309e-02]
 [ 1.59689011e+00]
 [-1.92571041e+00]]
rotation
6
Kalman:  [[-1.43167963e+09]
 [-1.43167963e+09]
 [-1.23114413e-02]
 [ 1.66370666e+00]
 [-1.68732953e+00]]
rotation
7
Kalman:  [[-1.43167963e+09]
 [-1.43167963e+09]
 [ 2.58333335e-02]
 [ 1.68922841e+00]
 [-1.59627579e

CancelledError: 

In [1]:
await stop_robot()

NameError: name 'stop_robot' is not defined